In [42]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data
data_path = "nurse_schedule_project2_data_large_VA.xlsx"
locations = pd.read_excel(data_path, sheet_name="locations", index_col = 0, header = None)
nurses = pd.read_excel(data_path, sheet_name="nurses", index_col = 0)
task_time = pd.read_excel(data_path, sheet_name="task_execution_time", index_col = 0).T

patients = pd.read_excel(data_path, sheet_name="patients", index_col = 0)
# Break comma separated strings into lists
for col in ['M', 'T', 'W','Th','F','S','Su']:
    patients[col] = patients[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
for col in ['skillset']:
    nurses[col] = nurses[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Transpose the dataframe to access patient information easily
patients = patients.T
nurses = nurses.T


# Sets and Parameters
N = nurses.columns.tolist()            # set of nurses
P = patients.columns.tolist()          # set of patients
L = list(locations.index)         # set of locations
D = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']  # set of days
T_tasks = task_time.columns.tolist() # set of tasks

# Parameters


# define a function to list all patients in a given location
def find_keys_with_inner_value(df, target_value):
    return list(df.T[df.T['location'] == target_value].index)

print(T_tasks)



['medication', 'drawing blood', 'physical therapy', 'wound care', 'administering injections', 'personal hygiene assistance']


In [43]:
# City with needs on each day,
# array of 7 lists, each list contains patients in that city on that day 

# total task time / city / day 

# function to determine total task time / city / day
def calculate_total_hours_per_city_per_day(day, city): 
    total_hours = 0
    for patient in find_keys_with_inner_value(patients, city):
        for task in patients[patient][day]:
            total_hours += task_time[task]
    return total_hours

# function to determine total task time / city / day / task
def calculate_total_hours_per_city_per_day_per_task(day, city, task): 
    total_hours = 0
    for patient in find_keys_with_inner_value(patients, city):
        for theTask in patients[patient][day]:
            if theTask == task:
                total_hours += task_time[task]
    return total_hours


# make an array for each nurse with the total hours they work on each day (init to 0)
eachNursesHours = {}
for nurse in N:
    eachNursesHours[nurse] = {}
    for day in D:
        eachNursesHours[nurse][day] = 0

# make an array for each city with the total hours of work needed on each day using the function above
eachCityHours = {}
for city in L:
    eachCityHours[city] = {}
    for day in D:
        eachCityHours[city][day] = {}
        for task in T_tasks:
            eachCityHours[city][day][task] = calculate_total_hours_per_city_per_day_per_task(day, city, task)

print(eachCityHours)
hoursAlexandriaMondayHygiene = calculate_total_hours_per_city_per_day_per_task('M', 'Alexandria', 'personal hygiene assistance')
print(hoursAlexandriaMondayHygiene)

{'Alexandria': {'M': {'medication': Time    40
Name: medication, dtype: int64, 'drawing blood': Time    89
Name: drawing blood, dtype: int64, 'physical therapy': 0, 'wound care': Time    150
Name: wound care, dtype: int64, 'administering injections': Time    40
Name: administering injections, dtype: int64, 'personal hygiene assistance': Time    216
Name: personal hygiene assistance, dtype: int64}, 'T': {'medication': Time    40
Name: medication, dtype: int64, 'drawing blood': Time    89
Name: drawing blood, dtype: int64, 'physical therapy': 0, 'wound care': Time    150
Name: wound care, dtype: int64, 'administering injections': Time    40
Name: administering injections, dtype: int64, 'personal hygiene assistance': Time    216
Name: personal hygiene assistance, dtype: int64}, 'W': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administering injections': 0, 'personal hygiene assistance': 0}, 'Th': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0